# <center>A study on pre-processing video images to improve the accuracy of hand tremor classifiers</center>

<center><em>Eduardo Furtado - Master's Dissertation</em></center>
<center><em>Advisor: Prof. Dr. Ana Cristina Bicharra Garcia</em></center>
<center><em>UNIRIO PPGI - Universidade Federal do Estado do Rio de Janeiro Graduate Program in Computer Science</em></center>

In this study, we aim to verify if pre-processing videos of patients performing finger tapping tasks can improve the outcome of pose estimation algorithms, and in turn, enhance the performance of a hand tremor classifier. Finger tapping videos are fundamental in clinical evaluations, aiding in determining the MDS-UPDRS score for Parkinson's patients. As deep neural networks play an increasing role in medical diagnostics, AI stands out as a valuable tool for the early diagnosis of movement disorders. Maintaining high video data quality is essential. Employing 2D videos emerges as a cost-effective and user-friendly strategy that enables remote medical assessments as simple as recording a video with a webcam. By drawing on established methodologies, we hope to provide a robust and clinically relevant approach.

Our research unfolds across two objectives, the first one is to reproduce and assess the methodology efficacy of the results presented by **Yang et al.** in `Automatic Detection Pipeline for Accessing the Motor Severity of Parkinson’s Disease in Finger Tapping and Postural Stability`, utilizing the PDMotorDB dataset for hand tremor sevirity classification based on their MDS-UPDRS score. The second objective is to explore the impact of pre-processing techniques to our data can improve the performance of the tremor severity classifier model. In the separate code for data preparation, we draw inspiration from a similar study by **Chen et al.** with their application of the Contrast Limited Adaptive Histogram Equalization (CLAHE) for pose estimation of bedridden patients in clinical conditions, we also perform data augmentation to our dataset by rotating and mirroring each video we have available. Finally, we'll also experiment new features in order to give more information from the extracted finger tapping signal to our classifier model.

PDMotorDB dataset: https://github.com/pddata/PDMotorDB


For the baseline of this study, we will replicate the methodology applied by Yang et al. to create all the domain knowledge features they used and train a DNN model to classify the severity of tremor from the hand videos. Hand landmarks for both the right and left hand and save in a csv file. 

**From the paper (page 5)**

```
In the domain knowledge extraction stage, we extract three features: the tapping rate, the tapping frozen times, and the tapping amplitude variation for the finger tapping action. Those features represent the patient’s ability to control their fingers. 

The tapping rate is defined as the tapping number divided by the time interval. 
Tapping frozen times is defined as the hesitation number of the patient. 
The tapping amplitude variation refers to the distance variation in which the patient spread the thumb and the index finger. 

To calculate the three features above, we first calculate the distance from the thumb to the index finger for each frame. Finger tapping status can be represented by the distances. Those distances compose of a discrete signal, which can form a wave form by connecting the adjacent point. To make the wave form smooth and filter noise, we use low pass filtering to get rid of the frequency with over 4 Hz. Second, we normalize the distance by dividing by the maximum distance to avoid the variation of the hand distance to the camera. For example, when the distance comes to zero, the thumb and the index finger are pressed together. When the distance comes to one, the thumb and the index finger are pulled apart at most. The wave form of one cycle can be viewed as a tapping, shown in Figure 3(a). The mark ‘x’ in yellow represents the local minimal point.

From the wave form, the tapping rate can be calculated as the tapping number divided by time. For example, in the first row of Figure 3(b), the tapping rate is the number of ‘x’ mark, the local minimal point divided by five seconds. The tapping rate is 1.6 taps per second in this example. The frozen state is defined as the patient prolonging the current action for a while. In the second row of Figure 3(b), we can see there is an obvious flat line in the center of the wave form. We first calculate the first derivative of the wave form and take the range with small derivatives as the frozen interval. We take the number of the frozen interval as the feature of tapping frozen times. The tapping amplitude refers to the patient’s ability to keep the tapping amplitude. For example, there is an obvious decrease in the amplitude of the wave form, shown in the third row of Figure 3(b). We calculate the standard deviation of the tapping amplitude to grasp the tapping amplitude variation, avoiding the effects of the various absolute pixel distances. Besides, this is also consistent with the criteria in the 3.12 part of MDS-UPDRS.
```

In [ ]:
import numpy as np
import pandas as pd

method = 'original'      # 'original' / 'clahe' / 'original_augmented'
selected_hand = 'L'      # 'L' / 'R'
new_features = False     # True / False

def load_data(method, selected_hand):
    if selected_hand == 'L':
        ranges = [(1, 50), (51, 100), (101, 150), (151, 200), (201, 250), (251, 300), (301, 370)]
    else:
        ranges = [(1, 50), (51, 100), (101, 150), (151, 200), (201, 250)]

    if method == 'original':
        path = r"D:\data\PDMotorDB\lr_keypoints_v1"
        df = pd.read_csv(path + fr"\data_keypoints_{selected_hand}.csv")
        return df

    elif method == 'clahe':
        path = r"D:\data\PDMotorDB\lr_clahe_keypoints_v1"
        df = pd.read_csv(path + fr"\data_keypoints_{selected_hand}.csv")
        return df

    elif method == 'original_augmented':
        path = r"D:\data\PDMotorDB\lr_keypoints_v1"
        df = pd.read_csv(path + fr"\data_keypoints_{selected_hand}.csv")

        path = r"D:\data\PDMotorDB\lr_augmented_keypoints_v1"
        for lower_than, greater_than in ranges:
            df_aux = pd.read_csv(path + fr"\data_keypoints_{selected_hand}_{lower_than}_{greater_than}.csv")
            df = pd.concat([df, df_aux])
        return df

df = load_data(method, selected_hand)
df.shape

In [ ]:
df.head()

In [ ]:
df['participant'] = df['Source'].str.replace('.avi', '', regex=False).str.split('_').str[0]

In [ ]:
df['dist'] = np.sqrt((df['thumb_tip_x'] - df['index_tip_x'])**2 + 
                     (df['thumb_tip_y'] - df['index_tip_y'])**2)

df.shape

## Visualizing the tapping signal

In [ ]:
import plotly.graph_objects as go

for source in df['Source'].unique():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df[df['Source']==source]['Frame'], y=df[df['Source']==source]['dist'], mode='markers+lines'))

    fig.update_layout(title=f'{source}').show()
    break

## Creating the dataset of features for each video

In [ ]:
import pywt
import numpy as np
from scipy.signal import find_peaks
from scipy.stats import entropy



list_features = ['tapping_rate', 'tapping_std', 'tapping_frozen']
new_features_list = ['skewness', 'kurtosis', 'autocorr', 'peak_count', 'signal_entropy', 'wavelet_energy', 'second_derivative']



if new_features:
    list_features += new_features_list



def compute_statistics(dists):
    skewness = dists.skew()
    kurtosis = dists.kurtosis()
    return skewness, kurtosis

def compute_autocorrelation(dists):
    autocorr = dists.autocorr()
    return autocorr


def compute_peak_features(dists):
    peaks, _ = find_peaks(dists)
    peak_count = len(peaks)
    return peak_count

def compute_entropy(dists):
    signal_entropy = entropy(np.histogram(dists)[0])
    return signal_entropy


def compute_wavelet_features(dists):
    coeffs = pywt.wavedec(dists, 'db1', level=2)
    cA2, cD2, cD1 = coeffs
    wavelet_energy = np.sum(cD2**2) + np.sum(cD1**2)
    return wavelet_energy

def compute_second_derivative(dists):
    second_derivative = np.diff(dists, n=2)
    return np.mean(second_derivative**2)


In [ ]:
from sklearn.preprocessing import minmax_scale
from scipy.signal import butter, filtfilt

video_length_seconds = 5

# low pass filter
cutoff = 4
sampling_rate = 25
order = 2
normal_cutoff = cutoff / (0.5 * sampling_rate)
b, a = butter(order, normal_cutoff, btype='low', analog=False)

def prepare_dataset(df):
    dataset = []

    #for participant in df['participant'].unique(): 
    for s in df['Source'].unique():             
        # print(s)
        data = df[(df['Source']==s)].copy()
        
        data['dist_filter'] = filtfilt(b, a, data['dist'])

        data['dist_norm'] = minmax_scale(data['dist_filter'])

        # local minima
        data['flag_minima_rolling'] = np.where(data['dist_norm'] == data['dist_norm'].rolling(5, center=True).min(), True, False)
        
        # tapping rate
        tapping_rate = data['flag_minima_rolling'].sum() / video_length_seconds

        # tapping frozen
        data['dist_derivative'] = data['dist_norm'].diff()

        frozen_threshold = 0.001

        data['frozen_state'] = np.abs(data['dist_derivative']) < frozen_threshold
        tapping_frozen = data[data['frozen_state']==True].shape[0]

        # tapping standard deviation
        tapping_std = data['dist_norm'].std()

        if new_features:
            # skewness and kurtosis
            skewness, kurtosis = compute_statistics(data['dist_norm'])

            # autocorrelation
            autocorr = compute_autocorrelation(data['dist_norm'])

            # peak count
            peak_count = compute_peak_features(data['dist_norm'])

            # entropy
            signal_entropy = compute_entropy(data['dist_norm'])

            # wavelet energy
            wavelet_energy = compute_wavelet_features(data['dist_norm'])

            # second derivative
            second_derivative = compute_second_derivative(data['dist_norm'])

            dataset.append({'source': s, 'participant': data['participant'].unique()[0], 'tapping_rate': tapping_rate, 'tapping_std': tapping_std, 'tapping_frozen': tapping_frozen, 'skewness': skewness, 'kurtosis': kurtosis, 'autocorr': autocorr, 'peak_count': peak_count, 'signal_entropy': signal_entropy, 'wavelet_energy': wavelet_energy, 'second_derivative': second_derivative})
        else:
            dataset.append({'source': s, 'participant': data['participant'].unique()[0], 'tapping_rate': tapping_rate, 'tapping_std': tapping_std, 'tapping_frozen': tapping_frozen})

    df_dataset = pd.DataFrame.from_records(dataset)
    df_dataset.shape

    return df_dataset

dataset = prepare_dataset(df)
print(dataset.shape)

In [ ]:
dataset.head()

## Reading the target values for each video in the dataset

In [ ]:
df_target = pd.DataFrame()

path_target = r"D:\data\PDMotorDB"

files = [path_target + "\\lefthand_train.txt", path_target + "\\lefthand_val.txt", 
         path_target + "\\righthand_train.txt", path_target + "\\righthand_val.txt"]

for file in files:
    participant_prefix = 'L' if 'lefthand' in file else 'R'

    with open(file, 'r') as f:
        lines = f.readlines()
        df_lines = pd.DataFrame(lines, columns=['participant'])
        
        df_lines['target'] = df_lines['participant'].str.replace('\n', '').str.split(' ').str[1].astype(int)
        df_lines['participant'] = participant_prefix + df_lines['participant'].str.split(' ').str[0]
        df_lines['train_val'] = 'train' if 'train' in file else 'val'

        #df_target = df_target.append(df_lines)
        df_target = pd.concat([df_target, df_lines])
df_target.shape

In [ ]:
df_target.head()

In [ ]:
dataset = dataset.merge(df_target, on='participant', how='left')

print(dataset.shape)

In [ ]:
# Removing dirty data from the dataset

exclude_ids = [ 'L00008', 'L00013', 'L00031', 'L00035', 'L00036', 'L00042', 'L00075', 'L00095', 'L00098', 'L00104', 'L00105', 'L00116', 'L00139', 'L00170', 'L00188', 'L00189', 'L00191', 'L00193', 'L00198', 'L00214', 'L00217', 'L00228', 'L00231', 'L00233', 'L00236', 'L00244', 'L00251', 'L00253', 'L00255', 'L00258', 'L00262', 'L00263', 'L00271', 'L00282', 'L00284', 'L00294', 'L00305', 'L00307', 'L00312', 'L00338', 'L00343', 'L00368', 'R00017', 'R00034', 'R00035', 'R00041', 'R00054', 'R00060', 'R00063', 'R00071', 'R00074', 'R00085', 'R00088', 'R00105', 'R00118', 'R00138', 'R00139', 'R00146', 'R00159', 'R00178', 'R00194', 'R00195', 'R00197', 'R00203', 'R00212', 'R00214', 'R00243',]

dataset = dataset[~dataset['participant'].isin(exclude_ids)]

print(dataset.shape)

In [ ]:
dataset.head()

## Visualizing data distribution for the features in our dataset

In [ ]:
import plotly.express as px

for feature in list_features:
    fig = px.histogram(dataset, x=feature, nbins=20, title=feature, marginal="box", hover_data=dataset.columns)
    fig.show()

## Creating the model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers

input_shape = (3 + len(new_features_list),) if new_features else (3,)
print(input_shape)

def create_model():
    model = tf.keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l2(1e-6)),
        layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l2(1e-6)),
        layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l2(1e-6)),
        layers.Dense(dataset['target'].nunique(), activation='softmax')               
    ])

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=10000,
        decay_rate=0.9)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                loss='categorical_crossentropy', 
                metrics=['accuracy',
                         tf.keras.metrics.Precision(),
                         tf.keras.metrics.Recall()])
    
    return model

In [ ]:
model = create_model()

## Train test split the dataset

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

train_dataset = dataset[(dataset['train_val']=='train') & (dataset['participant'].str.contains(selected_hand))]

test_dataset = dataset[(dataset['train_val']=='val') & (dataset['participant'].str.contains(selected_hand))]

X_train, X_val, y_train, y_val = train_test_split(train_dataset[list_features], train_dataset['target'], test_size=0.4, random_state=42)

print(f'''
Train shape: {X_train.shape} {y_train.shape} \t {np.unique(y_train, return_counts=True)}
Val shape:  {X_val.shape} {y_val.shape} \t {np.unique(y_val, return_counts=True)}
''')

y_train = to_categorical(y_train, num_classes=dataset['target'].nunique())
y_val = to_categorical(y_val, num_classes=dataset['target'].nunique())


In [ ]:
X_train.head()

In [ ]:
y_train[:5]

## Training the model

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200)

In [ ]:
import plotly.graph_objects as go

trace_train = go.Scatter(
    x=[i for i in range(len(history.history['loss']))],
    y=history.history['loss'],
    mode='lines', name='Train Loss'
)

trace_test = go.Scatter(
    x=[i for i in range(len(history.history['val_loss']))],
    y=history.history['val_loss'],
    mode='lines', name='Test Loss'
)

fig = go.Figure(data=[trace_train, trace_test]).update_layout(title=f'Train and Validation Loss over Epochs for <b>{selected_hand} hand</b>', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

## Performing 5-fold cross-validation to our data

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import datetime

# Define the number of folds
n_splits = 5 

# Create StratifiedKFold object
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Convert labels to categorical
y = to_categorical(dataset['target'], num_classes=dataset['target'].nunique())

# Prepare arrays to store results
precision_scores, recall_scores, f1_scores, model_filenames = [], [], [], []

# Loop over each fold
for train_index, val_index in skf.split(dataset[list_features], np.argmax(y, axis=1)):

    # Split data
    X_train, X_val = dataset.iloc[train_index][list_features], dataset.iloc[val_index][list_features]
    y_train, y_val = y[train_index], y[val_index]

    # Create and compile model
    model = create_model()

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, verbose=0)

    # Evaluate the model
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_val, axis=1)

    # Calculate metrics
    precision_scores.append(precision_score(y_true, y_pred_classes, average='macro'))
    recall_scores.append(recall_score(y_true, y_pred_classes, average='macro'))
    f1_scores.append(f1_score(y_true, y_pred_classes, average='macro'))

    output_path = r"D:\data\master_experiments"
    model_filename = f'\\model_{selected_hand}_{method}_new_features_{new_features}_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}.h5'
    model_filenames.append(model_filename)
    # print(model_filename)
    model.save(output_path + model_filename)

# Calculate average scores across all folds
average_precision = np.mean(precision_scores)
average_recall = np.mean(recall_scores)
average_f1 = np.mean(f1_scores)

# Print average scores
print(f'''
Average Precision: {average_precision}, 
Average Recall: {average_recall}, 
Average F1-Score: {average_f1}''')


## Saving experiment results

In [ ]:
experiment_summary = {
    'Hand': selected_hand, 
    'Experiment': method.title(), 
    'New Features': new_features,
    'Avg Precision': average_precision, 
    'Avg Recall': average_recall, 
    'Avg F1': average_f1,
    'Precision Scores': precision_scores,
    'Recall Scores': recall_scores,
    'F1 Scores': f1_scores,
    'Models': model_filenames
}

import json
print(json.dumps(experiment_summary, indent=4))

with open(output_path + f"\\model_{selected_hand}_{method}_new_features_{new_features}.json", 'w') as f:
    json.dump(experiment_summary, f)